<a href="https://colab.research.google.com/github/leedhn/capstone21-1/blob/main/run_mnli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import numpy as np
import pandas as pd

In [ ]:
#dir = '/content/drive/MyDrive/21-1_SoJong/20210407/'

#PATH = '.inhouse_split_test_qids.txt' #train.jsonl 에서 가져온듯
#CSQA_PATH ='train_rand_split.jsonl'
#PRED_PATH ='.predictions_test.csv'

In [ ]:
%cd '/content/drive/MyDrive/21-1_SoJong/20210407/'

/content/drive/MyDrive/21-1_SoJong/20210407


In [ ]:
!pip install fairseq

     |████████████████████████████████| 1.7MB 12.3MB/s 
     |████████████████████████████████| 133kB 54.0MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 112kB 46.5MB/s 
     |████████████████████████████████| 645kB 36.1MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=f070e7f356b8847143fab2c98d87af4c1d218b0c2d13fdd61cf1b814ce8d94a9
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#MNLI 돌리기

MNLI 함수 def

In [ ]:
import torch
from fairseq.data.data_utils import collate_tokens

class Contradiction:
    def __init__(self):

        self.dir=''
        self.txt=[]
        self.batches=[] #Roberta에 넣을 batch 사이즈로 txt를 분할한 리스트
        self.result_list=[] # result들 txt파일별로 묶기 위해 만듦
        self.result=[] #Roberta prediction 결과 저장
        self.contradiction_list=[] #txt파일별로 cotradiction이 나온 pair 저장
        self.roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli') #사용할 모델
        self.roberta.cuda() #gpu사용
        self.roberta.eval()

    #txt파일 batch사이즈로 분할해서 self.batches에 저장
    def make_batch(self,size):
        self.batches = [self.txt[i*size:(i+1)*size] for i in range((len(self.txt)+size-1)//size)]
        #return self.batches

    # roberta 돌려서 contradiction 모으기 (메인 기능)
    def count_contradict(self):

        self.result_list=[]
        self.contradiction_list=[]
        #0=contradiction
        #1=natural
        #2=entailment

        k=0
        for batch_of_pairs in self.batches:
            
            self.batch = collate_tokens(
                [self.roberta.encode(pair[0], pair[1]) for pair in batch_of_pairs], pad_idx=1
            )

            logprobs = self.roberta.predict('mnli', self.batch)
            self.result=logprobs.argmax(dim=1)
            #print(k,'th batch')
            #print(result)
            self.result_list.extend(self.result)
            for r in range(len(self.result)):
                if self.result[r]==0:
                    self.contradiction_list.append(batch_of_pairs[r])
                    
            k+=1
           
        #return (self.result_list, self.contradiction_list)

    # 디렉토리에서 파일 읽어서 self.txt에 저장
    def get_txt(self,dir): #txt 형식은 'ground truth' \t 'predict' \n ...
        self.txt=[]
        with open(dir,'r') as f:
            lines = f.readlines()
            for line in lines: 
                self.txt.append(line[:-1].split('\t'))
        #print(self.txt[:5][1])
        #return self.txt

    # 외부에서 클래스 함수 돌릴 때 사용. return값 있음
    def run(self,dir,size):
        self.get_txt(dir)
        print('run file dir:',dir)
        self.make_batch(size)
        self.count_contradict()
        len_contrad=len(self.contradiction_list)
        len_txt=len(self.txt)
        print(len_contrad)
        #print(self.result_list)
        print('file ',dir,'is done')
        return (len_contrad,len_txt,(len_contrad/len_txt), self.contradiction_list, self.result_list)

MNLI 돌리기


In [ ]:
contrad_file={} #여기에 파일 별 결과들이 저장됩니다. key는 파일 이름. value는 결과값들.

batch_size=16 #추후에 수정 가능

#filename_list=['oracle_cont','masked100_cont']
filename_list=['test', 'dev', 'train']

file_type='txt'
file_dir='/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input'

model=Contradiction()

for name in filename_list:
    dir= file_dir+'/'+name+'.'+file_type
    print(dir)
    contrad_file[name]=model.run(dir,batch_size)
    # [contradiction 개수, txt 전체 개수, 비율, contradiction 전문, roberta prediction 리스트]

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip
100%|██████████| 751652118/751652118 [01:06<00:00, 11254957.40B/s]
1042301B [00:01, 1040114.29B/s]
456318B [00:00, 620567.14B/s]


/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/test.txt
run file dir: /content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/test.txt
6361
file  /content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/test.txt is done
/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/dev.txt
run file dir: /content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/dev.txt
7103
file  /content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/dev.txt is done
/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/train.txt
run file dir: /content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/train.txt
54883
file  /content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/train.txt is done


In [ ]:
###### 수습 (mnli_input/txt 파일들 길이 반씩으로 자르기)
# import os
# filename_list=['test.txt', 'dev.txt', 'train.txt']
# new_file_list = ['new_test.txt', 'new_dev.txt', 'new_train.txt']
# file_dir='/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input'
# for i in range(len(filename_list)):
#   dir = os.path.join(file_dir, filename_list[i])
#   new_dir = os.path.join(file_dir, new_file_list[i])
#   with open(dir) as f: 
#     path_list = f.read().splitlines()
#   path_list = path_list[len(path_list)//2:]
#   print(len(path_list))
#   with open(new_dir, 'a') as f:
#     f.write('\n'.join(path_list)) 

28500
30525
243525


mnli 결과 -1 핸들링 하려고 만든 데이터프레임들

df_test, df_train, df_dev --> except_dump_xxx.csv 로 저장됨

---
mnli 결과 -1 핸들링 하기 위한 덤프 인덱스 목록

train_dump, test_dump, dev_dump 

---
원래 mnli 결과 그대로 가지는 데이터 프레임들

train_result, test_result, dev_result -->xxx_result.csv 로 저장됨

In [ ]:
import pandas as pd

# 0 : contradiction 1 : neutral 2 : entailment
df_test = pd.DataFrame(contrad_file['test'][4], columns = ['result'])
df_test['result'] = df_test['result'].apply(int)
df_train = pd.DataFrame(contrad_file['train'][4], columns = ['result'])
df_train['result'] = df_train['result'].apply(int)
df_dev = pd.DataFrame(contrad_file['dev'][4], columns = ['result'])
df_dev['result'] = df_dev['result'].apply(int)


dump 인덱스 목록 불러오기

In [ ]:
import pickle

with open ('/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/dump_train_idx.txt','rb') as f:
    train_dump = pickle.load(f)
with open ('/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/dump_test_idx.txt','rb') as f:
    test_dump = pickle.load(f)
with open ('/content/drive/MyDrive/21-1_SoJong/20210407/mnli_input/dump_dev_idx.txt','rb') as f:
    dev_dump = pickle.load(f)

In [ ]:
for i in train_dump:
    df_train['result'][i]=-1
for i in test_dump:
    df_test['result'][i]=-1
for i in dev_dump:
    df_dev['result'][i]=-1 

In [ ]:
df_dev['result'][28]  

-1

In [ ]:
input_dir = '/content/drive/MyDrive/21-1_SoJong/20210407/mnli_result/'
# train
train_result = pd.DataFrame(contrad_file['train'][4], columns = ['result'])
train_result['result'] = train_result['result'].apply(int)
train_result.to_csv(input_dir + 'train_result.csv', index = False)
df_train.to_csv(input_dir + 'except_dump_train_result.csv', index=False)

# test
test_result = pd.DataFrame(contrad_file['test'][4], columns = ['result'])
test_result['result'] = test_result['result'].apply(int)
test_result.to_csv(input_dir + 'test_result.csv', index = False)
df_test.to_csv(input_dir + 'except_dump_test_result.csv', index=False)

# dev
dev_result = pd.DataFrame(contrad_file['dev'][4], columns = ['result'])
dev_result['result'] = dev_result['result'].apply(int)
dev_result.to_csv(input_dir + 'dev_result.csv', index = False)
df_dev.to_csv(input_dir + 'except_dump_dev_result.csv', index=False)

In [ ]:
#### 수습 : 길이 반으로 자르기
# dir = '/content/drive/MyDrive/21-1_SoJong/20210407/mnli_result'
# train_result = pd.read_csv(os.path.join(dir, 'train_result.csv'))
# train_result = train_result.iloc[len(train_result) // 2:]
# df_train = pd.read_csv(os.path.join(dir, 'except_dump_train_result.csv'))
# df_train = df_train.iloc[len(df_train) // 2:]
# train_result.to_csv(os.path.join(dir,'train_result.csv'), index = False)
# df_train.to_csv(os.path.join(dir,'except_dump_train_result.csv'), index=False)

# dev_result = pd.read_csv(os.path.join(dir, 'dev_result.csv'))
# dev_result = dev_result.iloc[len(dev_result) // 2:]
# df_dev = pd.read_csv(os.path.join(dir, 'except_dump_dev_result.csv'))
# df_dev = df_dev.iloc[len(df_dev) // 2:]
# dev_result.to_csv(os.path.join(dir,'dev_result.csv'), index = False)
# df_dev.to_csv(os.path.join(dir,'except_dump_dev_result.csv'), index=False)

# test_result = pd.read_csv(os.path.join(dir, 'test_result.csv'))
# test_result = test_result.iloc[len(test_result) // 2:]
# df_test = pd.read_csv(os.path.join(dir, 'except_dump_test_result.csv'))
# df_test = df_test.iloc[len(df_test) // 2:]
# test_result.to_csv(os.path.join(dir,'test_result.csv'), index = False)
# df_test.to_csv(os.path.join(dir,'except_dump_test_result.csv'), index=False)

# # check
# train_result = pd.read_csv(os.path.join(dir, 'train_result.csv'))
# df_train = pd.read_csv(os.path.join(dir, 'except_dump_train_result.csv'))
# dev_result = pd.read_csv(os.path.join(dir, 'dev_result.csv'))
# df_dev = pd.read_csv(os.path.join(dir, 'except_dump_dev_result.csv'))
# test_result = pd.read_csv(os.path.join(dir, 'test_result.csv'))
# df_test = pd.read_csv(os.path.join(dir, 'except_dump_test_result.csv'))

In [ ]:
print(len(train_result), len(df_train))
print(len(test_result), len(df_test))
print(len(dev_result), len(df_dev))

243525 243525
28500 28500
30525 30525
